In [ ]:
# NFL Draft Prediction - 特徴量エンジニアリング

## 概要
EDAで得られた知見を基に、予測性能向上のための特徴量エンジニアリングを実施します。

## 目標
- ベースラインスコア0.80792を上回る
- ドメイン知識を活用した有効な特徴量作成
- 欠損値処理の最適化
- ポジション特化の特徴量作成


In [ ]:
## 1. セットアップ


In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings

# 設定
warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option('display.max_columns', None)

print("ライブラリの読み込み完了")


In [ ]:
## 2. データ読み込み


In [ ]:
# データ読み込み
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission_6_20修正.csv')

print(f"Train: {train.shape}, Test: {test.shape}")

# データのコピーを作成（元データを保持）
train_fe = train.copy()
test_fe = test.copy()

# 全データを結合（特徴量エンジニアリングのため）
all_data = pd.concat([train_fe, test_fe], ignore_index=True)
print(f"結合データ: {all_data.shape}")


In [ ]:
## 3. 基本特徴量の作成


In [ ]:
# 基本的な組み合わせ特徴量の作成
def create_basic_features(df):
    \"\"\"基本的な特徴量を作成\"\"\"
    df_new = df.copy()
    
    # 1. BMI（Body Mass Index）
    df_new['BMI'] = df_new['Weight'] / (df_new['Height'] ** 2)
    
    # 2. パワー・ウェイト・レシオ
    df_new['Power_Weight_Ratio'] = df_new['Bench_Press_Reps'] / df_new['Weight']
    
    # 3. スピード・パワー・インデックス（垂直跳び / 40ヤード走）
    df_new['Speed_Power_Index'] = df_new['Vertical_Jump'] / df_new['Sprint_40yd']
    
    # 4. アジリティ・スコア（3cone + shuttle）
    df_new['Agility_Score'] = df_new['Agility_3cone'] + df_new['Shuttle']
    
    # 5. 爆発力指標（立ち幅跳び * 垂直跳び）
    df_new['Explosiveness'] = df_new['Broad_Jump'] * df_new['Vertical_Jump']
    
    # 6. スピード×爆発力
    df_new['Speed_Explosiveness'] = df_new['Explosiveness'] / df_new['Sprint_40yd']
    
    # 7. 体格指標（身長×体重）
    df_new['Size_Index'] = df_new['Height'] * df_new['Weight']
    
    # 8. ベンチプレス効率（回数/体重×身長）
    df_new['Bench_Efficiency'] = df_new['Bench_Press_Reps'] / df_new['Size_Index']
    
    # 9. 年齢グループ（カテゴリ化）
    df_new['Age_Group'] = pd.cut(df_new['Age'], bins=[0, 21, 23, 30], 
                                labels=['Young', 'Standard', 'Veteran'], 
                                include_lowest=True)
    
    print(\"基本特徴量作成完了\")\n    return df_new

# 基本特徴量の作成
all_data_fe = create_basic_features(all_data)

# 新特徴量の確認
new_features = ['BMI', 'Power_Weight_Ratio', 'Speed_Power_Index', 'Agility_Score', 
                'Explosiveness', 'Speed_Explosiveness', 'Size_Index', 'Bench_Efficiency']

print(f\"\\n=== 新特徴量の統計 ===\")
print(all_data_fe[new_features].describe())


In [ ]:
## 4. ポジション特化特徴量の作成


In [ ]:
# ポジション特化の特徴量作成
def create_position_features(df):\n    \"\"\"ポジション別の特徴量を作成\"\"\"\n    df_new = df.copy()\n    \n    # ポジションタイプ別の正規化特徴量\n    numerical_cols = ['Height', 'Weight', 'Sprint_40yd', 'Vertical_Jump', \n                     'Bench_Press_Reps', 'Broad_Jump', 'Agility_3cone', 'Shuttle']\n    \n    for col in numerical_cols:\n        if col in df_new.columns:\n            # ポジションタイプ別の平均値で正規化\n            position_means = df_new.groupby('Position_Type')[col].transform('mean')\n            df_new[f'{col}_vs_Position_Mean'] = df_new[col] / position_means\n            \n            # ポジションタイプ別の標準偏差で正規化\n            position_stds = df_new.groupby('Position_Type')[col].transform('std')\n            df_new[f'{col}_Position_Zscore'] = (df_new[col] - position_means) / position_stds\n    \n    # ポジション特化の組み合わせ特徴量\n    # 1. オフェンシブライン用特徴量\n    offensive_line_mask = df_new['Position_Type'] == 'offensive_lineman'\n    df_new['OL_Strength_Index'] = 0\n    df_new.loc[offensive_line_mask, 'OL_Strength_Index'] = (\n        df_new.loc[offensive_line_mask, 'Bench_Press_Reps'] * \n        df_new.loc[offensive_line_mask, 'Weight'] / 100\n    )\n    \n    # 2. スキルポジション用特徴量\n    skill_positions = ['backs_receivers']\n    skill_mask = df_new['Position_Type'].isin(skill_positions)\n    df_new['Skill_Speed_Index'] = 0\n    df_new.loc[skill_mask, 'Skill_Speed_Index'] = (\n        df_new.loc[skill_mask, 'Vertical_Jump'] * \n        df_new.loc[skill_mask, 'Broad_Jump'] / \n        df_new.loc[skill_mask, 'Sprint_40yd']\n    )\n    \n    # 3. ディフェンス用特徴量\n    defense_positions = ['defensive_lineman', 'line_backer', 'defensive_back']\n    defense_mask = df_new['Position_Type'].isin(defense_positions)\n    df_new['Defense_Athleticism'] = 0\n    df_new.loc[defense_mask, 'Defense_Athleticism'] = (\n        (df_new.loc[defense_mask, 'Vertical_Jump'] + \n         df_new.loc[defense_mask, 'Broad_Jump']) / \n        (df_new.loc[defense_mask, 'Sprint_40yd'] + \n         df_new.loc[defense_mask, 'Agility_3cone'])\n    )\n    \n    print(\"ポジション特化特徴量作成完了\")\n    return df_new\n\n# ポジション特化特徴量の作成\nall_data_fe = create_position_features(all_data_fe)\n\n# ポジション特化特徴量の確認\nposition_features = ['OL_Strength_Index', 'Skill_Speed_Index', 'Defense_Athleticism']\nprint(f\"\\n=== ポジション特化特徴量の統計 ===\")\nprint(all_data_fe[position_features].describe())


In [ ]:
## 5. 学校プレステージ特徴量の作成


In [ ]:
# 学校プレステージ特徴量の作成\ndef create_school_features(df):\n    \"\"\"学校に関する特徴量を作成\"\"\"\n    df_new = df.copy()\n    \n    # 訓練データのみを使って学校統計を計算（データリークを防ぐ）\n    train_data = df_new[df_new['Drafted'].notna()].copy()\n    \n    # 学校別の指名率と選手数\n    school_stats = train_data.groupby('School').agg({\n        'Drafted': ['count', 'mean']\n    })\n    school_stats.columns = ['School_Player_Count', 'School_Draft_Rate']\n    \n    # 5人未満の学校は統計が不安定なので平均値で補完\n    overall_draft_rate = train_data['Drafted'].mean()\n    school_stats.loc[school_stats['School_Player_Count'] < 5, 'School_Draft_Rate'] = overall_draft_rate\n    \n    # 学校プレステージカテゴリ\n    def categorize_school_prestige(draft_rate, player_count):\n        if pd.isna(draft_rate):\n            return 'Unknown'\n        elif draft_rate >= 0.8:\n            return 'Elite'\n        elif draft_rate >= 0.6:\n            return 'High'\n        elif draft_rate >= 0.4:\n            return 'Medium'\n        else:\n            return 'Low'\n    \n    school_stats['School_Prestige'] = school_stats.apply(\n        lambda x: categorize_school_prestige(x['School_Draft_Rate'], x['School_Player_Count']), axis=1\n    )\n    \n    # 全データにマージ\n    df_new = df_new.merge(school_stats, left_on='School', right_index=True, how='left')\n    \n    # 新しい学校（テストデータのみに存在）は平均値で補完\n    df_new['School_Draft_Rate'].fillna(overall_draft_rate, inplace=True)\n    df_new['School_Player_Count'].fillna(1, inplace=True)\n    df_new['School_Prestige'].fillna('Unknown', inplace=True)\n    \n    print(\"学校プレステージ特徴量作成完了\")\n    return df_new\n\n# 学校特徴量の作成\nall_data_fe = create_school_features(all_data_fe)\n\n# 学校プレステージの分布確認\nprint(f\"\\n=== 学校プレステージ分布 ===\")\nprint(all_data_fe['School_Prestige'].value_counts())\nprint(f\"\\n=== 学校指名率統計 ===\")\nprint(all_data_fe['School_Draft_Rate'].describe())


In [ ]:
## 6. 欠損値処理の最適化


In [ ]:
# 改良された欠損値処理\ndef advanced_missing_value_handling(df):\n    \"\"\"ポジション別・パターン別の欠損値処理\"\"\"\n    df_new = df.copy()\n    \n    # 身体能力テスト系の列\n    performance_cols = ['Sprint_40yd', 'Vertical_Jump', 'Bench_Press_Reps', \n                       'Broad_Jump', 'Agility_3cone', 'Shuttle']\n    \n    # 1. 欠損パターンフラグの作成\n    for col in performance_cols:\n        df_new[f'{col}_is_missing'] = df_new[col].isnull().astype(int)\n    \n    # 2. 欠損値の総数\n    df_new['Total_Missing_Tests'] = df_new[[f'{col}_is_missing' for col in performance_cols]].sum(axis=1)\n    \n    # 3. ポジション別の中央値で補完\n    for col in performance_cols:\n        if col in df_new.columns:\n            # ポジションタイプ別の中央値\n            position_medians = df_new.groupby('Position_Type')[col].median()\n            \n            for position in position_medians.index:\n                mask = (df_new['Position_Type'] == position) & (df_new[col].isnull())\n                df_new.loc[mask, col] = position_medians[position]\n            \n            # 残った欠損値は全体の中央値で補完\n            overall_median = df_new[col].median()\n            df_new[col].fillna(overall_median, inplace=True)\n    \n    # 4. Age の補完\n    if 'Age' in df_new.columns:\n        # ポジション別の中央値で補完\n        age_medians = df_new.groupby('Position_Type')['Age'].median()\n        for position in age_medians.index:\n            mask = (df_new['Position_Type'] == position) & (df_new['Age'].isnull())\n            df_new.loc[mask, 'Age'] = age_medians[position]\n        \n        # 残った欠損値は全体の中央値で補完\n        df_new['Age'].fillna(df_new['Age'].median(), inplace=True)\n    \n    print(\"改良された欠損値処理完了\")\n    return df_new\n\n# 欠損値処理の実行\nall_data_fe = advanced_missing_value_handling(all_data_fe)\n\n# 欠損値処理後の確認\nprint(f\"\\n=== 欠損値処理後の状況 ===\")\nmissing_after = all_data_fe.isnull().sum()\nprint(missing_after[missing_after > 0])\n\nif len(missing_after[missing_after > 0]) == 0:\n    print(\"すべての欠損値が処理されました！\")"


In [ ]:
## 7. カテゴリ変数のエンコーディング


In [ ]:
# カテゴリ変数のエンコーディング\ndef encode_categorical_features(df):\n    \"\"\"カテゴリ変数の効果的なエンコーディング\"\"\"\n    df_new = df.copy()\n    \n    # Label Encodingする列\n    label_encode_cols = ['School', 'Player_Type', 'Position_Type', 'Position', \n                        'Age_Group', 'School_Prestige']\n    \n    encoders = {}\n    for col in label_encode_cols:\n        if col in df_new.columns:\n            le = LabelEncoder()\n            df_new[col] = le.fit_transform(df_new[col].astype(str))\n            encoders[col] = le\n    \n    # 年度の処理（既に数値なのでそのまま）\n    # 年度の周期性を考慮した特徴量\n    if 'Year' in df_new.columns:\n        df_new['Year_sin'] = np.sin(2 * np.pi * (df_new['Year'] - df_new['Year'].min()) / \n                                   (df_new['Year'].max() - df_new['Year'].min()))\n        df_new['Year_cos'] = np.cos(2 * np.pi * (df_new['Year'] - df_new['Year'].min()) / \n                                   (df_new['Year'].max() - df_new['Year'].min()))\n    \n    print(\"カテゴリ変数エンコーディング完了\")\n    return df_new, encoders\n\n# エンコーディングの実行\nall_data_fe, label_encoders = encode_categorical_features(all_data_fe)\n\n# エンコーディング後のデータタイプ確認\nprint(f\"\\n=== エンコーディング後のデータタイプ ===\")\nprint(all_data_fe.dtypes.value_counts())\nprint(f\"\\n=== 数値以外の列（残っている場合） ===\")\nnon_numeric = all_data_fe.select_dtypes(exclude=[np.number]).columns\nprint(list(non_numeric))


In [ ]:
## 8. 最終的な特徴量選択とデータ分割


In [ ]:
# 最終的な特徴量選択とデータ分割\ndef prepare_final_dataset(df, train_len):\n    \"\"\"最終的なデータセットの準備\"\"\"\n    # 使用しない列を除外\n    exclude_cols = ['Id', 'Drafted']\n    feature_cols = [col for col in df.columns if col not in exclude_cols]\n    \n    # 数値型以外の列があれば確認\n    non_numeric_cols = df[feature_cols].select_dtypes(exclude=[np.number]).columns\n    if len(non_numeric_cols) > 0:\n        print(f\"警告: 数値型以外の列があります: {list(non_numeric_cols)}\")\n        # 強制的に数値型に変換\n        for col in non_numeric_cols:\n            df[col] = pd.to_numeric(df[col], errors='coerce')\n    \n    # 無限大や極端に大きな値の処理\n    for col in feature_cols:\n        if col in df.columns:\n            # 無限大を NaN に変換\n            df[col] = df[col].replace([np.inf, -np.inf], np.nan)\n            \n            # 残った NaN を中央値で補完\n            if df[col].isnull().sum() > 0:\n                df[col].fillna(df[col].median(), inplace=True)\n    \n    # データを分割\n    train_processed = df[:train_len].copy()\n    test_processed = df[train_len:].copy()\n    \n    # 特徴量とターゲットを分離\n    X_train = train_processed[feature_cols]\n    y_train = train_processed['Drafted']\n    X_test = test_processed[feature_cols]\n    \n    print(f\"特徴量数: {len(feature_cols)}\")\n    print(f\"訓練データサイズ: {X_train.shape}\")\n    print(f\"テストデータサイズ: {X_test.shape}\")\n    print(f\"欠損値確認 - 訓練: {X_train.isnull().sum().sum()}, テスト: {X_test.isnull().sum().sum()}\")\n    \n    return X_train, y_train, X_test, feature_cols\n\n# 最終データセットの準備\nX_train_fe, y_train_fe, X_test_fe, feature_cols_fe = prepare_final_dataset(all_data_fe, len(train))\n\n# 作成された特徴量の一覧\nprint(f\"\\n=== 作成された特徴量一覧（総数: {len(feature_cols_fe)}） ===\")\nfor i, col in enumerate(feature_cols_fe, 1):\n    print(f\"{i:2d}. {col}\")"


In [ ]:
## 9. 改良モデルの構築・評価


In [ ]:
# 改良されたRandomForestモデルの評価
def evaluate_improved_model(X_train, y_train, n_splits=5, random_state=42):
    \"\"\"改良されたモデルの交差検証評価\"\"\"\n    \n    # Stratified K-Fold設定\n    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)\n    \n    # スコア格納用\n    cv_scores = []\n    feature_importance_sum = np.zeros(X_train.shape[1])\n    \n    print(\"=== 5-Fold交差検証実行中 ===\")\n    \n    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):\n        # データ分割\n        X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]\n        y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]\n        \n        # モデル訓練（ハイパーパラメータ最適化）\n        rf_model = RandomForestClassifier(\n            n_estimators=200,\n            max_depth=10,\n            min_samples_split=10,\n            min_samples_leaf=5,\n            max_features='sqrt',\n            class_weight='balanced',\n            random_state=random_state + fold,\n            n_jobs=-1\n        )\n        \n        rf_model.fit(X_fold_train, y_fold_train)\n        \n        # 予測・評価\n        y_pred_proba = rf_model.predict_proba(X_fold_val)[:, 1]\n        fold_score = roc_auc_score(y_fold_val, y_pred_proba)\n        cv_scores.append(fold_score)\n        \n        # 特徴量重要度の累積\n        feature_importance_sum += rf_model.feature_importances_\n        \n        print(f\"Fold {fold}: ROC AUC = {fold_score:.5f}\")\n    \n    # 結果の集計\n    mean_score = np.mean(cv_scores)\n    std_score = np.std(cv_scores)\n    feature_importance_avg = feature_importance_sum / n_splits\n    \n    print(f\"\\n=== 最終結果 ===\")\n    print(f\"平均ROC AUC: {mean_score:.5f} ± {std_score:.5f}\")\n    print(f\"ベースラインとの比較: {mean_score:.5f} vs 0.80792\")\n    \n    if mean_score > 0.80792:\n        improvement = ((mean_score - 0.80792) / 0.80792) * 100\n        print(f\"🎉 ベースライン改善: +{improvement:.2f}%\")\n    else:\n        decline = ((0.80792 - mean_score) / 0.80792) * 100\n        print(f\"⚠️  ベースライン下回り: -{decline:.2f}%\")\n    \n    return cv_scores, feature_importance_avg\n\n# 改良モデルの評価実行\ncv_scores_fe, feature_importance_fe = evaluate_improved_model(X_train_fe, y_train_fe)\n\n# 特徴量重要度の可視化\nfeature_importance_df = pd.DataFrame({\n    'feature': feature_cols_fe,\n    'importance': feature_importance_fe\n}).sort_values('importance', ascending=False)\n\nprint(f\"\\n=== 特徴量重要度TOP20 ===\")\nprint(feature_importance_df.head(20))"


In [ ]:
# 特徴量重要度の可視化
plt.figure(figsize=(12, 10))
top_20_features = feature_importance_df.head(20)
sns.barplot(data=top_20_features, y='feature', x='importance', palette='viridis')
plt.title('特徴量重要度 TOP20（RandomForest）')
plt.xlabel('重要度')
plt.ylabel('特徴量')
plt.tight_layout()
plt.show()

# 新規作成特徴量の評価
engineered_features = ['BMI', 'Power_Weight_Ratio', 'Speed_Power_Index', 'Agility_Score', 
                      'Explosiveness', 'Speed_Explosiveness', 'Size_Index', 'Bench_Efficiency',
                      'OL_Strength_Index', 'Skill_Speed_Index', 'Defense_Athleticism',
                      'School_Draft_Rate', 'Total_Missing_Tests']

engineered_importance = feature_importance_df[feature_importance_df['feature'].isin(engineered_features)]
print(f"\n=== 新規作成特徴量の重要度 ===")
print(engineered_importance.sort_values('importance', ascending=False))

# 新規特徴量の総合貢献度
total_engineered_importance = engineered_importance['importance'].sum()
total_importance = feature_importance_df['importance'].sum()
engineered_contribution = (total_engineered_importance / total_importance) * 100
print(f"\n新規特徴量の総合貢献度: {engineered_contribution:.1f}%")


In [ ]:
## 10. 最終予測と提出ファイル作成


In [ ]:
# 最終モデルの訓練とテストデータ予測
def create_final_submission(X_train, y_train, X_test, test_ids, random_state=42):
    """最終的な提出ファイルを作成"""
    
    print("=== 最終モデル訓練中 ===")
    
    # 最終モデルの設定（全訓練データで学習）
    final_model = RandomForestClassifier(
        n_estimators=300,  # 少し増やしてみる
        max_depth=12,
        min_samples_split=8,
        min_samples_leaf=4,
        max_features='sqrt',
        class_weight='balanced',
        random_state=random_state,
        n_jobs=-1
    )
    
    # 全訓練データで学習
    final_model.fit(X_train, y_train)
    
    # テストデータで予測
    test_predictions = final_model.predict_proba(X_test)[:, 1]
    
    # 提出ファイル作成
    submission = pd.DataFrame({
        'Id': test_ids,
        'Drafted': test_predictions
    })
    
    # ファイル保存
    submission_filename = 'improved_submission_with_feature_engineering.csv'
    submission.to_csv(submission_filename, index=False)
    
    print(f"提出ファイル保存完了: {submission_filename}")
    print(f"予測値の統計:")
    print(f"  平均: {test_predictions.mean():.4f}")
    print(f"  中央値: {np.median(test_predictions):.4f}")
    print(f"  最小値: {test_predictions.min():.4f}")
    print(f"  最大値: {test_predictions.max():.4f}")
    print(f"  標準偏差: {test_predictions.std():.4f}")
    
    return submission, final_model

# 最終提出ファイルの作成
test_ids = test['Id'].values
final_submission, final_model = create_final_submission(X_train_fe, y_train_fe, X_test_fe, test_ids)

# 予測分布の可視化
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(final_submission['Drafted'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
plt.title('テストデータ予測値の分布')
plt.xlabel('予測された指名確率')
plt.ylabel('頻度')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
training_predictions = final_model.predict_proba(X_train_fe)[:, 1]
plt.hist(training_predictions, bins=50, alpha=0.7, color='lightcoral', edgecolor='black', label='訓練データ')
plt.hist(final_submission['Drafted'], bins=50, alpha=0.7, color='skyblue', edgecolor='black', label='テストデータ')
plt.title('予測値分布の比較')
plt.xlabel('予測された指名確率')
plt.ylabel('頻度')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n最終提出ファイル: {final_submission.head()}")


In [ ]:
# 特徴量重要度の可視化
plt.figure(figsize=(12, 10))\ntop_20_features = feature_importance_df.head(20)\nsns.barplot(data=top_20_features, y='feature', x='importance', palette='viridis')\nplt.title('特徴量重要度 TOP20（RandomForest）')\nplt.xlabel('重要度')\nplt.ylabel('特徴量')\nplt.tight_layout()\nplt.show()\n\n# 新規作成特徴量の評価\nengineered_features = ['BMI', 'Power_Weight_Ratio', 'Speed_Power_Index', 'Agility_Score', \n                      'Explosiveness', 'Speed_Explosiveness', 'Size_Index', 'Bench_Efficiency',\n                      'OL_Strength_Index', 'Skill_Speed_Index', 'Defense_Athleticism',\n                      'School_Draft_Rate', 'Total_Missing_Tests']\n\nengineered_importance = feature_importance_df[feature_importance_df['feature'].isin(engineered_features)]\nprint(f\"\\n=== 新規作成特徴量の重要度 ===\")\nprint(engineered_importance.sort_values('importance', ascending=False))\n\n# 新規特徴量の総合貢献度\ntotal_engineered_importance = engineered_importance['importance'].sum()\ntotal_importance = feature_importance_df['importance'].sum()\nengineered_contribution = (total_engineered_importance / total_importance) * 100\nprint(f\"\\n新規特徴量の総合貢献度: {engineered_contribution:.1f}%\")


In [ ]:
## 10. 最終予測と提出ファイル作成


In [ ]:
# 最終モデルの訓練とテストデータ予測
def create_final_submission(X_train, y_train, X_test, test_ids, random_state=42):\n    \"\"\"最終的な提出ファイルを作成\"\"\"\n    \n    print(\"=== 最終モデル訓練中 ===\")\n    \n    # 最終モデルの設定（全訓練データで学習）\n    final_model = RandomForestClassifier(\n        n_estimators=300,  # 少し増やしてみる\n        max_depth=12,\n        min_samples_split=8,\n        min_samples_leaf=4,\n        max_features='sqrt',\n        class_weight='balanced',\n        random_state=random_state,\n        n_jobs=-1\n    )\n    \n    # 全訓練データで学習\n    final_model.fit(X_train, y_train)\n    \n    # テストデータで予測\n    test_predictions = final_model.predict_proba(X_test)[:, 1]\n    \n    # 提出ファイル作成\n    submission = pd.DataFrame({\n        'Id': test_ids,\n        'Drafted': test_predictions\n    })\n    \n    # ファイル保存\n    submission_filename = 'improved_submission_with_feature_engineering.csv'\n    submission.to_csv(submission_filename, index=False)\n    \n    print(f\"提出ファイル保存完了: {submission_filename}\")\n    print(f\"予測値の統計:\")\n    print(f\"  平均: {test_predictions.mean():.4f}\")\n    print(f\"  中央値: {np.median(test_predictions):.4f}\")\n    print(f\"  最小値: {test_predictions.min():.4f}\")\n    print(f\"  最大値: {test_predictions.max():.4f}\")\n    print(f\"  標準偏差: {test_predictions.std():.4f}\")\n    \n    return submission, final_model\n\n# 最終提出ファイルの作成\ntest_ids = test['Id'].values\nfinal_submission, final_model = create_final_submission(X_train_fe, y_train_fe, X_test_fe, test_ids)\n\n# 予測分布の可視化\nplt.figure(figsize=(12, 5))\n\nplt.subplot(1, 2, 1)\nplt.hist(final_submission['Drafted'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')\nplt.title('テストデータ予測値の分布')\nplt.xlabel('予測された指名確率')\nplt.ylabel('頻度')\nplt.grid(True, alpha=0.3)\n\nplt.subplot(1, 2, 2)\ntraining_predictions = final_model.predict_proba(X_train_fe)[:, 1]\nplt.hist(training_predictions, bins=50, alpha=0.7, color='lightcoral', edgecolor='black', label='訓練データ')\nplt.hist(final_submission['Drafted'], bins=50, alpha=0.7, color='skyblue', edgecolor='black', label='テストデータ')\nplt.title('予測値分布の比較')\nplt.xlabel('予測された指名確率')\nplt.ylabel('頻度')\nplt.legend()\nplt.grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.show()\n\nprint(f\"\\n最終提出ファイル: {final_submission.head()}\")


In [ ]:
## 11. まとめと次のステップ


In [ ]:
# 改善結果の総括\nprint(\"=== 特徴量エンジニアリング結果の総括 ===\")\nprint(f\"1. 総特徴量数: {len(feature_cols_fe)}\")\nprint(f\"2. 交差検証スコア: {np.mean(cv_scores_fe):.5f} ± {np.std(cv_scores_fe):.5f}\")\nprint(f\"3. ベースライン比較: {np.mean(cv_scores_fe):.5f} vs 0.80792\")\n\nif np.mean(cv_scores_fe) > 0.80792:\n    improvement = ((np.mean(cv_scores_fe) - 0.80792) / 0.80792) * 100\n    print(f\"4. 改善度: +{improvement:.2f}%\")\nelse:\n    decline = ((0.80792 - np.mean(cv_scores_fe)) / 0.80792) * 100\n    print(f\"4. 改善度: -{decline:.2f}%\")\n\nprint(f\"\\n=== 作成した主要特徴量 ===\")\nprint(\"- 基本特徴量: BMI, Power_Weight_Ratio, Speed_Power_Index など\")\nprint(\"- ポジション特化特徴量: 各ポジション用の専用指標\")\nprint(\"- 学校プレステージ特徴量: 学校の指名実績に基づく特徴量\")\nprint(\"- 欠損値パターン特徴量: 欠損の情報も特徴量として活用\")\nprint(\"- ポジション別正規化特徴量: ポジション内での相対的な能力\")\n\nprint(f\"\\n=== 次のステップ（さらなる改善案） ===\")\nprint(\"1. LightGBM/XGBoostなどの勾配ブースティング手法の試行\")\nprint(\"2. ハイパーパラメータの詳細最適化（Optuna等）\")\nprint(\"3. アンサンブル手法（複数モデルの組み合わせ）\")\nprint(\"4. より高度な特徴量選択手法\")\nprint(\"5. 年度別の時系列分割検証\")\nprint(\"6. ポジション特化モデルの構築\")\n\nprint(f\"\\n=== ファイル出力 ===\")\nprint(f\"- 改良版提出ファイル: improved_submission_with_feature_engineering.csv\")\nprint(f\"- 特徴量重要度が保存され、分析に活用可能\")
